In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 设置路径
train_feature_dir = '../2025_A2/train/Features'
train_metadata_path = '../2025_A2/train/train_metadata.csv'

In [3]:

# 读取特征数据
color_df = pd.read_csv(os.path.join(train_feature_dir, 'color_histogram.csv'))
hog_df = pd.read_csv(os.path.join(train_feature_dir, 'hog_pca.csv'))
additional_df = pd.read_csv(os.path.join(train_feature_dir, 'additional_features.csv'))

def merge(meta):
    return (meta
            .merge(additional_df,  on="image_path")
            .merge(color_df, on="image_path")
            .merge(hog_df,    on="image_path"))

metadata_df = pd.read_csv(train_metadata_path)

# 合并特征
X = merge(metadata_df).drop(columns=["ClassId", "image_path"])

print(X.dtypes)  # 确保所有列都是 float 或 int 类型

id                int64
edge_density    float64
mean_b          float64
mean_g          float64
mean_r          float64
                 ...   
hog_pca_15      float64
hog_pca_16      float64
hog_pca_17      float64
hog_pca_18      float64
hog_pca_19      float64
Length: 121, dtype: object


In [4]:
# 读取标签
metadata_df = pd.read_csv(train_metadata_path)
y = metadata_df['ClassId']
y_encoded = to_categorical(y, num_classes=43)

# 特征标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集与验证集
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.1, random_state=42)

In [5]:
# 搭建神经网络模型
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(43, activation='softmax')  # 43类
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=64)

Epoch 1/25
78/78 [==============================] - 1s 4ms/step - loss: 3.2147 - accuracy: 0.1796 - val_loss: 2.3411 - val_accuracy: 0.4372
Epoch 2/25
78/78 [==============================] - 0s 3ms/step - loss: 2.1005 - accuracy: 0.4141 - val_loss: 1.4586 - val_accuracy: 0.5519
Epoch 3/25
78/78 [==============================] - 0s 3ms/step - loss: 1.5388 - accuracy: 0.5224 - val_loss: 1.1151 - val_accuracy: 0.6630
Epoch 4/25
78/78 [==============================] - 0s 3ms/step - loss: 1.2245 - accuracy: 0.6127 - val_loss: 0.9317 - val_accuracy: 0.6995
Epoch 5/25
78/78 [==============================] - 0s 3ms/step - loss: 1.0904 - accuracy: 0.6441 - val_loss: 0.8662 - val_accuracy: 0.7250
Epoch 6/25
78/78 [==============================] - 0s 3ms/step - loss: 0.9443 - accuracy: 0.6876 - val_loss: 0.7778 - val_accuracy: 0.7486
Epoch 7/25
78/78 [==============================] - 0s 3ms/step - loss: 0.8388 - accuracy: 0.7222 - val_loss: 0.7311 - val_accuracy: 0.7596
Epoch 8/25
78/78 [==

In [6]:
# 评估验证集准确率
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"\nValidation Accuracy: {val_acc:.4f}")

18/18 [==============================] - 0s 886us/step - loss: 0.5013 - accuracy: 0.8251

Validation Accuracy: 0.8251


In [8]:
# 保存模型（可选）
model.save('../models/neural_model.h5')

In [23]:
feature_dir_test = '../2025_A2/test/Features'
meta_path_test = '../2025_A2/test/test_metadata.csv'

color_df_test = pd.read_csv(f"{feature_dir_test}/color_histogram.csv")
hog_df_test = pd.read_csv(f"{feature_dir_test}/hog_pca.csv")
additional_df_test = pd.read_csv(f"{feature_dir_test}/additional_features.csv")
def merge_test(meta):
    return (meta
            .merge(additional_df_test,  on="image_path")
            .merge(color_df_test, on="image_path")
            .merge(hog_df_test,    on="image_path"))

metadata_df_test = pd.read_csv(meta_path_test)

X_test = merge_test(metadata_df_test).drop(columns=["ClassId", "image_path"])

X_test_scaled = scaler.fit_transform(X_test)

proba_nn_test = model.predict(X_test_scaled)
y_pred_test = np.argmax(proba_nn_test, axis=1)
test_ids = pd.read_csv(meta_path_test)['id']
submission = pd.DataFrame({
    'id': test_ids,
    'ClassId': y_pred_test
})
submission.to_csv('../nn.csv', index=False)

74/74 [==============================] - 0s 762us/step
